In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model as lm
from statsmodels.sandbox.regression.gmm import IV2SLS 
from statsmodels.sandbox.regression.gmm import GMM

# PART A
1.	Download the “dataset_lm.csv” file from Canvas and upload it to Jupyter Notebook. 
2.	Run the OLS model by using the dependent and explanatory variables in the dataset. 
3.	Show your summary table in Python and interpret your results in the summary report.


In [26]:
df = pd.read_csv('dataset_lm.csv')

In [27]:
df

,Dependent Var,Explanatory Var #1,Explanatory Var #2,Explanatory Var #3,Explanatory Var #4,Explanatory Var #5,Explanatory Var #6,Explanatory Var #7,Explanatory Var #8,Explanatory Var #9,Explanatory Var #10,Explanatory Var #11,Explanatory Var #12,Explanatory Var #13,Explanatory Var #14,Explanatory Var #15
0,56.293458,13.698667,50.639873,0,-18.568035,45.121911,11.412501,56.410757,2,-12.281132,38.996909,-3.010548,49.195073,0,-21.153143,46.919314
1,58.473431,2.714725,65.845845,1,-25.105932,47.190213,10.080280,65.383107,3,-36.763585,51.654939,4.991111,45.591729,0,-6.474403,53.383508
2,94.195330,11.618072,65.072497,0,-7.897464,52.163036,11.057301,82.812717,0,-15.733547,48.913837,-2.457696,56.608806,0,-27.903299,48.515026
3,29.074583,0.818623,45.408996,1,-18.316132,54.356714,5.029029,48.812471,1,-12.825591,45.851732,14.974177,47.362594,1,-10.064411,55.266254
4,86.035569,9.077544,73.548021,0,-19.204165,47.186807,12.128134,62.520911,2,-13.804860,47.765904,9.593982,53.700562,0,-17.546302,48.150543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,61.300432,0.338441,70.430598,0,-21.525979,50.741115,-8.050843,39.075397,3,-29.197173,57.473862,15.505202,73.280605,1,-3.602482,56.317474
418,26.309237,-1.729712,47.087996,1,-19.034807,55.242929,28.001769,76.429649,0,-21.941401,54.641620,14.295688,49.816798,1,-13.910494,52.827988
419,58.350627,18.322301,53.267835,0,-26.186201,36.702958,14.530116,51.275150,2,-20.699670,46.443687,5.708762,52.751016,1,-6.202345,52.467289
420,31.954003,0.436357,61.844132,0,-25.881961,57.106851,21.786066,68.928442,0,-23.166461,59.194840,8.365762,64.291056,1,-21.330017,50.237673


In [28]:
y = df["Dependent Var"]
X = df.drop("Dependent Var", axis=1)
X = sm.add_constant(X)

ols_model = sm.OLS(y, X).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:          Dependent Var   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.325e+30
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        15:55:14   Log-Likelihood:                 12332.
No. Observations:                 422   AIC:                        -2.463e+04
Df Residuals:                     406   BIC:                        -2.457e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  32.0000    

# PART B
1.	Use error values from the OLS model to calculate their standard deviation and autocorrelation values for the first three lags. 
2.	Then, run the GLS model accordingly. 
3.	Show your summary table in Python and interpret your results in the summary report.

In [29]:
# Calculate residuals
ols_residuals = ols_model.resid

# Calculate standard deviation of residuals
ols_residuals_std = ols_residuals.std()

autocorrelation_lag1 = np.corrcoef(ols_residuals[:-1], ols_residuals[1:])[0, 1]
autocorrelation_lag2 = np.corrcoef(ols_residuals[:-2], ols_residuals[2:])[0, 1]
autocorrelation_lag3 = np.corrcoef(ols_residuals[:-3], ols_residuals[3:])[0, 1]

print(f"Standard Deviation of OLS Residuals: {ols_residuals_std}")
print("Autocorrelation for the first three lags:")
print(f"Lag 1: {autocorrelation_lag1}")
print(f"Lag 2: {autocorrelation_lag2}")
print(f"Lag 3: {autocorrelation_lag3}")


Standard Deviation of OLS Residuals: 4.933401227123459e-14
Autocorrelation for the first three lags:
Lag 1: 0.06780015960053946
Lag 2: 0.017936447749035474
Lag 3: 0.05468134594634336


In [30]:
from statsmodels.regression.linear_model import GLS

# GLS model with the specified autocorrelation structure
gls_model = GLS(y, X, sigma=ols_residuals_std, rho=[autocorrelation_lag1, autocorrelation_lag2, autocorrelation_lag3])
gls_results = gls_model.fit()

print(gls_results.summary())

                            GLS Regression Results                            
Dep. Variable:          Dependent Var   R-squared:                       1.000
Model:                            GLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.495e+30
Date:                Mon, 23 Oct 2023   Prob (F-statistic):               0.00
Time:                        15:55:14   Log-Likelihood:                 12229.
No. Observations:                 422   AIC:                        -2.443e+04
Df Residuals:                     406   BIC:                        -2.436e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  32.0000   5

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:130: ValueWarning: unknown kwargs ['rho']
  warnings.warn(msg, ValueWarning)


# Part C
1.	Split the dataset into two as the training and test sets (test size = 0.5). 
2.	Run the Lasso model with alpha=1 and estimate the coefficients using the training set. 
3.	Then, calculate the mean absolute percentage error using the test set. 
4.	Find an approximate value for alpha that minimizes the mean absolute percentage error.


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

#Lasso model with alpha=1
lasso_model = Lasso(alpha=1)
lasso_model.fit(X_train, y_train)

lasso_coefficients = lasso_model.coef_

print("Lasso Coefficients:")
for i, coef in enumerate(lasso_coefficients):
    print(f"Explanatory Var #{i+1}: {coef:.4f}")

Lasso Coefficients:
Explanatory Var #1: 0.0000
Explanatory Var #2: 1.2697
Explanatory Var #3: 1.6839
Explanatory Var #4: 2.0263
Explanatory Var #5: 2.0876
Explanatory Var #6: -0.9175
Explanatory Var #7: -0.0000
Explanatory Var #8: 0.0000
Explanatory Var #9: -0.0000
Explanatory Var #10: 0.0000
Explanatory Var #11: -0.0000
Explanatory Var #12: 0.0131
Explanatory Var #13: 0.0000
Explanatory Var #14: -0.0000
Explanatory Var #15: 0.0000
Explanatory Var #16: -0.0362


In [32]:
from sklearn.metrics import mean_absolute_error

lasso_predictions = lasso_model.predict(X_test)

absolute_percentage_errors = abs((y_test - lasso_predictions) / y_test) * 100
mape = absolute_percentage_errors.mean()
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))

Mean Absolute Percentage Error (MAPE): 4.43%


In [33]:
from sklearn.linear_model import LassoCV

# random range of alpha values to search for
alphas = np.logspace(-4, 4, 9) 

# Cross-validate
lasso_cv_model = LassoCV(alphas=alphas, cv=5) 
lasso_cv_model.fit(X_train, y_train)

optimal_alpha = lasso_cv_model.alpha_

lasso_optimal_model = Lasso(alpha=optimal_alpha)
lasso_optimal_model.fit(X_train, y_train)

lasso_optimal_predictions = lasso_optimal_model.predict(X_test)
mape_optimal = mean_absolute_error(y_test, lasso_optimal_predictions)

print("Optimal Alpha which minimizes the MAPE:", optimal_alpha)
print("MAPE with Optimal Alpha: {:.2f}%".format(mape_optimal))


Optimal Alpha which minimizes the MAPE: 0.0001
MAPE with Optimal Alpha: 0.00%


# PART D
1.	Use the demand data given in the table and develop an appropriate forecasting model (i.e., the tailored regularization discussed in the class—see your slides for more info) that exploits the available information given in the table as much as possible.
2.	Interpret your results.


In [20]:
from scipy.optimize import minimize
from sklearn.metrics import r2_score
import statsmodels.api as sm

data = [
    [100, 71],
    [112, 30],
    [107, 75],
    [103, 64],
    [91, 41],
    [85, 51],
    [84, 42],
    [85, 51],
    [79, 57],
    [81, 49],
    [134, 134],
    [86, 52],
    [99, 99],
    [89, 56],
    [111, 81],
    [114, 79],
    [118, 73],
    [163, 163],
    [193, 193],
    [143, 99],
    [144, 91],
    [202, 202],
    [158, 105],
    [160, 101],
    [144, 96]
]

data = np.array(data)

Y = data[1:, 0]  # Demand
X = data[:-1, 0]  # Demand (t-1)
L = data[1:, 1]  # Advanced Demand

# Ordinary Least Squares (OLS) model
X_OLS = np.vstack((np.ones(len(X)), X)).T
beta_ols = np.linalg.lstsq(X_OLS, Y, rcond=None)[0]

# Optimization Problem
def objective_function(beta, Y, X, L):
    forecasted_demand = X.dot(beta)
    return np.linalg.norm(Y - forecasted_demand)

def constraint_function(beta, X, L):
    return X.dot(beta) - L

# constraints of the objective function
constraints = {'type': 'ineq', 'fun': constraint_function, 'args': (X, L)}

initial_guess = np.zeros(X.shape[0])

# using minimize function to solve objective function
result = minimize(objective_function, initial_guess, args=(Y, X, L), constraints=constraints)

# Calculate R-squared for Baseline OLS
X_OLS = sm.add_constant(X)
model_ols = sm.OLS(Y, X_OLS)
result_ols = model_ols.fit()
forecasted_demand_ols = result_ols.predict(X_OLS)
r2_baseline = r2_score(Y, forecasted_demand_ols)

# decide which demand to use based on lambda values. Use forecasted demand when lambda is negative. Use Advanced Demand (L) when lambda is positive
lambda_value = result.fun
if lambda_value < 0:
    forecast_source = forecasted_demand_ols
else:
    forecast_source = L 

# Calculate the forecasted demand and R-squared for Tailored Regularization
X_prime = np.column_stack((X, forecast_source))
X_prime = sm.add_constant(X_prime)
model_tailored = sm.OLS(Y, X_prime)
result_tailored = model_tailored.fit()
forecasted_demand = result_tailored.predict(X_prime)

SSE = np.sum((Y - forecasted_demand) ** 2)
SST = np.sum((Y - np.mean(Y)) ** 2)
r_squared = 1 - (SSE / SST)

if lambda_value < 0:
    print("Chosen Demand Source: Forecasted Demand")
else:
    print("Chosen Demand Source: Advanced Demand (L)")

print(f"R-squared Baseline OLS: {r2_baseline}")
print(f"R-squared (Tailored Regularization): {r_squared:.4f}")


Chosen Demand Source: Advanced Demand (L)
R-squared Baseline OLS: 0.504550248064856
R-squared (Tailored Regularization): 0.9123
